In [1]:
#pip install --upgrade bitsandbytes

In [ ]:
!pip install nsfw

In [2]:
from PIL import Image
from transformers import MllamaForConditionalGeneration,MllamaProcessor
from transformers import BitsAndBytesConfig
import torch
import requests

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                       # or load_in_4bit=True
    llm_int8_enable_fp32_cpu_offload=True,   # CPU로 일부 레이어 offload
)

model = MllamaForConditionalGeneration.from_pretrained(
  'Bllossom/llama-3.2-Korean-Bllossom-AICA-5B',
  quantization_config=quant_config,
  torch_dtype=torch.bfloat16,
  device_map='auto' 
)
processor = MllamaProcessor.from_pretrained('Bllossom/llama-3.2-Korean-Bllossom-AICA-5B')

C:\Users\b1005\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.40s/it]


In [9]:
save_directory = "./llama-3.2-Korean-Bllossom-AICA-5B_Q4"
model.save_pretrained(save_directory)
processor.save_pretrained(save_directory)

[]

In [3]:
image_path = '미타니아카리사진집.jpg'

In [4]:
image = Image.open(f'{image_path}').convert('RGB')

In [5]:
messages = [
  {'role': 'user','content': [
    {'type':'image'},
    {'type': 'text','text': '이 이미지에 대해서 설명해줘'}
    ]},
  ]
input_text = processor.tokenizer.apply_chat_template(
  messages,
  tokenize=False,
  add_generation_prompt=True
)

In [6]:
inputs = processor(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to(model.device)

output = model.generate(**inputs, max_new_tokens=256,temperature=0.1,eos_token_id=processor.tokenizer.convert_tokens_to_ids('<|eot_id|>'),use_cache=False)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>이 이미지에 대해서 설명해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

이 이미지는 샤워실에서 몸을 감싸 안고 있는 한 여성의 모습을 담았으며, 그녀의 얼굴과 몸 전체를 완전히 노출하도록 촬영한 것이 특징이에요.<|eot_id|>


In [7]:
# 2~3분 정도 걸림